## Fig 3: Halo model power spectra 

In [ ]:
# Standard imports
import sys
import numpy as np
import matplotlib.pyplot as plt

# Third-party imports
import camb # Calculates the linear matter power spectrum

# Halo Model library imports
sys.path.append('../halomodel/')
import cosmology # TODO: Remove dependancy
import halomodel

In [ ]:
# Set cosmological parameters
Omega_c = 0.25
Omega_b = 0.05
Omega_k = 0.0
h = 0.7
As = 1.97448e-9
ns = 0.96
w = -1.0
wa = 0.0
m_nu = 0.0 # in eV

# You can choose to set sigma_8, in that case we scale the power spectrum
sigma_8_set = True # if true uses the following value
sigma_8_in  = 0.7

# Colours
col_lin = 'black'
col_mat = 'C0'
col_gal = 'C1'
col_mg  = 'C4'

# Line styles
ls_li = '-'
ls_hm = '-'
ls_2h = '--'
ls_1h = ':'

# Labels
klab = r'$k\,/\,h \mathrm{Mpc}^{-1}$'

In [ ]:
# k range [h/Mpc]
kmin = 1e-3; kmax = 200
nk = 101
ks = np.logspace(np.log10(kmin), np.log10(kmax), nk)

# Starting redshift
z = 0.
zmax = 2.

# Redshift
zs = [1., 0.] # CAMB reorders from high-z to low-z, so we define it like this from the start to avoid confusion

In [ ]:
# Sets cosmological parameters in camb to calculate the linear power spectrum
pars = camb.CAMBparams()
wb   = Omega_b*h**2
wc   = Omega_c*h**2

# This function sets standard and helium set using BBN consistency
pars.set_cosmology(ombh2=wb, omch2=wc, H0=100.*h, mnu=m_nu, omk=Omega_k)
pars.set_dark_energy(w=w, wa=wa, dark_energy_model='ppf') 
pars.InitPower.set_params(As=As, ns=ns, r=0)
pars.set_matter_power(redshifts=zs, kmax=kmax) # Setup the linear matter power spectrum

# extract parameters from CAMB
Omega_m  = pars.omegam 
Omega_nu = pars.omeganu
Omega_L  = 1. + Omega_k - Omega_m
camb_results = camb.get_results(pars)
sigma_8 = (camb_results.get_sigma8()[zs.index(z)]).item()

if sigma_8_set:
#     sigma_8_original = sigma_8
    scaling = (sigma_8_in/sigma_8)**2
    As *= scaling
    pars.InitPower.set_params(As=As, ns=ns, r=0)
    

Pk_lin = camb.get_matter_power_interpolator(pars, 
                                            nonlinear=False, 
                                            hubble_units=True, 
                                            k_hunit=True, 
                                            kmax=kmax,
                                            var1=camb.model.Transfer_tot,
                                            var2=camb.model.Transfer_tot, 
                                            zmax=zmax,
                                           )
Pk_lin = Pk_lin.P # Single out the linear P(k) interpolator
camb_results = camb.get_results(pars)
sigma_8 = (camb_results.get_sigma8()[zs.index(z)]).item()

In [ ]:
# Initialise halo model
hmod = halomodel.halo_model(z, Omega_m, hm='Tinker et al. (2010)', Dv=330.)

# Calculate the matter density
print('Mean matter density [log10((Msun/h)/(Mpc/h)^-3)]:', np.log10(hmod.rhom))

In [ ]:
# # Halo mass range [Msun/h]
# Mmin = 1e8; Mmax = 1e16
# nM = 1025
# Ms = np.logspace(np.log10(Mmin), np.log10(Mmax), nM)

# To speed things we use these settings if you want more precision use the above numbers
# Halo mass range [Msun/h]
Mmin = 1e9; Mmax = 1e15; nM = 129
Ms = np.logspace(np.log10(Mmin), np.log10(Mmax), nM)

# Lagrangian radii corresponding to halo masses; 
Rs = hmod.Lagrangian_radius(Ms)

In [ ]:
# Get sigma(R) brute force
# sigmaRs_brute = utility.get_sigmaR(Rs,Pk_lin=lambda k: Pk_lin(0, k),kmin=1e-5,kmax=1e5,nk=1e5,integration_type='brute')

# Get sigma(R) using quad integration. Ignore the warning it is pretty accurate
# sigmaRs_quad = utility.get_sigmaR(Rs,Pk_lin=lambda k: Pk_lin(0, k),integration_type='quad')
# sigmaRs = sigmaRs_quad

sigmaRs_camb = cosmology.sigmaR(Rs, camb_results, integration_type='camb')
sigmaRs = sigmaRs_camb

In [ ]:
# Halo density profiles or window functions
Uk = np.zeros((nM, nk))
for iM, M in enumerate(Ms):
    c = halomodel.concentration(M, z, method='Duffy et al. (2008)', halo_definition='Mvir') # Specifiy 'Mvir' for Duffy c(M)
    rv = hmod.virial_radius(M) # Take Dv from the halomodel to ensure consistency
    for ik, k in enumerate(ks):
        Uk[iM, ik] = halomodel.halo_window_function(k, rv, c, profile='NFW')
        
# Initialise profile class
matter_profile = halomodel.halo_profile(ks, Ms, Ms, Uk, hmod.rhom, mass=True)  # Need mass=True here

In [ ]:
# Simple (unrealistic) HOD model
def HOD(M, Mmin=1e12, Msat=1e13):
    if M < Mmin:
        return 0.
    elif Mmin <= M and M < Msat:
        return 1.
    else:
        #return np.rint(M/Msat)
        return M/Msat

# Fill an array with galaxy number at each halo mass
Ng = np.zeros(nM)
for iM, M in enumerate(Ms):
    Ng[iM] = HOD(M)
    
# Compute the mean galaxy density corresponding to our HOD
rhog = hmod.average(Ms, Ng, sigmas=sigmaRs[-1])
print('Mean galaxy density [(Mpc/h)^-3]:', rhog)

# Plot HOD
plt.loglog(Ms, Ng, color=col_gal)
plt.xlabel('Halo mass [$h^{-1}\,M_\odot$]')
plt.ylabel('Number of galaxies')
plt.ylim(bottom=1e-1)
plt.xlim((1e11, 1e16))
plt.show()

In [ ]:
# Halo density profiles or window functions
Uk = np.zeros((nM, nk))
for iM, M in enumerate(Ms):
    c = halomodel.concentration(M, z, method='Duffy et al. (2008)', halo_definition='Mvir') # Specifiy 'Mvir' for Duffy c(M)
    rv = hmod.virial_radius(M) # Take Dv from the halomodel to ensure consistency
    for ik, k in enumerate(ks):
        Uk[iM, ik] = halomodel.halo_window_function(k, rv, c, profile='NFW')
        
# Initialise profile class
# Need discrete=True here because profile is of a discrete tracer
galaxy_profile = halomodel.halo_profile(ks, Ms, Ng, Uk, rhog, discrete=True)

In [ ]:
# Calculate the halo-model power spectrum
profiles = [matter_profile, galaxy_profile]
Pk_2h, Pk_1h, Pk_hm = hmod.power_spectrum(ks, Ms, profiles, lambda k: Pk_lin(z, k), sigmas=sigmaRs[-1])#, shot=True)

In [ ]:
# Plot matter, galaxy and cross spectra

# Axis limits
Pkmin = 1e1; Pkmax = 4e4
kmin_plot = 1e-3; kmax_plot = 1e1
rmin = 1e-1; rmax = 1e2
smin = 0.; smax = 2.4

# Initialise plot
#fig=plt.figure(figsize=(8,5), dpi= 100, facecolor='w', edgecolor='k')
plt.subplots(3, 1, figsize=(5,7), dpi= 100, sharex=True)
plt.subplots_adjust(wspace=0.1, hspace=0.1) 

# Lists for plots
Pks = [Pk_2h[0, 0, :], Pk_1h[0, 0, :], Pk_hm[0, 0, :], Pk_hm[1, 1, :], Pk_hm[0, 1, :]]
cols = 3*[col_mat]+[col_gal, col_mg]
lss = [ls_2h, ls_1h, ls_hm, ls_hm, ls_hm]
labs = [None, None, 'matter', 'galaxy', 'matter-galaxy']

# P(k)
plt.subplot(3, 1, 1)
plt.loglog(ks, Pk_lin(z, ks), color=col_lin, ls=ls_li, label='linear')
for (ls, lab) in zip([ls_2h, ls_1h], ['2-halo term', '1-halo term']):
    plt.plot(np.nan, ls=ls, label=lab, color='black')
for (Pk, col, ls, lab) in zip(Pks, cols, lss, labs):
    plt.loglog(ks, Pk, color=col, ls=ls, label=lab)
plt.xticks([])
plt.xlim((kmin_plot, kmax_plot))
plt.ylabel(r'$P_{\rm uv}(k)\,/\,(h^{-1} \mathrm{Mpc})^3$')
plt.ylim((Pkmin, Pkmax))
plt.legend(ncol=2, loc='lower left', fontsize='9')

# Lists for plots
Pks = [Pk_2h[0, 0, :], Pk_1h[0, 0, :], Pk_hm[0, 0, :],
       Pk_2h[1, 1, :], Pk_1h[1, 1, :], Pk_hm[1, 1, :],
       Pk_2h[0, 1, :], Pk_1h[0, 1, :], Pk_hm[0, 1, :]]
cols = 3*[col_mat]+3*[col_gal]+3*[col_mg]
lss = 3*[ls_2h, ls_1h, ls_hm]

# Residual with linear
plt.subplot(3, 1, 2)
plt.loglog(ks, Pk_lin(z, ks)/Pk_lin(z, ks), color=col_lin)
for (Pk, col, ls) in zip(Pks, cols, lss):
    plt.loglog(ks, Pk/Pk_lin(z, ks), color=col, ls=ls)
plt.xticks([])
plt.xlim((kmin_plot, kmax_plot))
plt.ylabel(r'$P_{\rm uv}(k)\,/\,P^\mathrm{lin}(k)$')
plt.ylim((rmin, rmax))

# Residual with halo-model matter power
plt.subplot(3, 1, 3)
plt.semilogx(ks, Pk_lin(z, ks)/Pk_hm[0, 0, :], color=col_lin)
for (Pk, col, ls) in zip(Pks, cols, lss):
    plt.semilogx(ks, Pk/Pk_hm[0, 0, :], color=col, ls=ls)
plt.xlabel(klab)
plt.xlim((kmin_plot, kmax_plot))
plt.ylabel(r'$P_{\rm uv}(k)\,/\,P_\mathrm{mm}(k)$')
plt.ylim((smin, smax))

# Finish
#plt.tight_layout()
plt.savefig('plots/power_HOD_all.pdf', bbox_inches='tight')
plt.show()